In [ ]:
# Install dependencies
!pip install diffusers transformers accelerate safetensors flask flask-cors

In [ ]:
# Anime Image API
from flask import Flask, request, send_file
from flask_cors import CORS
from diffusers import StableDiffusionPipeline
import torch
import uuid
import os

# Flask app
app = Flask(__name__)
CORS(app)

# Load anime model (Waifu Diffusion)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = StableDiffusionPipeline.from_pretrained(
    'hakurei/waifu-diffusion',
    torch_dtype=torch.float16
)
pipe = pipe.to(device)

# Output folder
output_dir = 'outputs'
os.makedirs(output_dir, exist_ok=True)

@app.route('/generate', methods=['GET'])
def generate():
    prompt = request.args.get('prompt')
    if not prompt:
        return '❌ Error: prompt query required', 400

    output_path = os.path.join(output_dir, f'{uuid.uuid4()}.png')

    # Generate anime image
    image = pipe(prompt, guidance_scale=7.5).images[0]
    image.save(output_path)

    return send_file(output_path, mimetype='image/png')

# Run Flask app
app.run(host='0.0.0.0', port=7860)